In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
weather_data=pd.read_csv("weather_data.csv",encoding="utf-8")

In [34]:
print(weather_data.head(10))
weather_data.isnull().any()

          ﻿日期     天气状况        气温              风力风向
0   2015年1月1日     阴 /阴   8℃ / 6℃   东北风 ≤3级 /南风 ≤3级
1   2015年1月2日   多云 /多云  12℃ / 6℃    南风 ≤3级 /南风 ≤3级
2   2015年1月3日   多云 /多云  13℃ / 7℃    南风 ≤3级 /南风 ≤3级
3   2015年1月4日    多云 /阴  13℃ / 7℃  东北风 ≤3级 /东南风 ≤3级
4   2015年1月5日   小雨 /小雨  11℃ / 3℃  东南风 ≤3级 /东北风 ≤3级
5   2015年1月6日   小雨 /小雨   6℃ / 3℃  东北风 ≤3级 /东北风 ≤3级
6   2015年1月7日    阴 /小雨   6℃ / 2℃  东北风 ≤3级 /东北风 ≤3级
7   2015年1月8日   小雨 /小雨   5℃ / 1℃  东北风 ≤3级 /东北风 ≤3级
8   2015年1月9日  雨夹雪 /小雪   4℃ / 0℃  东北风 ≤3级 /东北风 ≤3级
9  2015年1月10日   小雨 /小雨   6℃ / 3℃  东北风 ≤3级 /东北风 ≤3级


﻿日期     False
天气状况    False
气温      False
风力风向    False
dtype: bool

In [35]:
weather_data.rename(columns={weather_data.columns[0]:'accidenttime'},inplace=True)

In [36]:
weather_data["accidenttime"]=weather_data["accidenttime"].str.replace("年","-")
weather_data["accidenttime"]=weather_data["accidenttime"].str.replace("月","-")
weather_data["accidenttime"]=weather_data["accidenttime"].str.replace("日","")

In [37]:
weather_data.rename(columns={weather_data.columns[1]:'weather'},inplace=True)
weather_data.rename(columns={weather_data.columns[2]:'temperature'},inplace=True)

In [38]:
temperature_max=weather_data["temperature"].str.extract(r"(\d*)℃",expand=False)
temperature_min=weather_data["temperature"].str.extract(r"/ (-*\d*)℃",expand=False)
weather_data["temperature_min"]=temperature_min
weather_data["temperature_max"]=temperature_max
# print(max_temp)

In [39]:
weather_data["weather"]=weather_data["weather"].str.strip()
weather_data["weather"]=weather_data["weather"].str.replace("~"," /")
weather1=weather_data["weather"].str.extract(r"(\S+) /",expand=False)
weather2=weather_data["weather"].str.extract(r"/(\S+)",expand=False)
#print(weather1[342:])
#print(weather2[342:])
error1=list(weather1[weather1.isnull()==True].index)
weather1[error1]=weather_data["weather"][error1]

error2=list(weather1[weather2.isnull()==True].index)
weather2[error2]=weather_data["weather"][error2]

weather_data["weather1"]=weather1
weather_data["weather2"]=weather2

In [40]:
wind=weather_data["风力风向"]
wind=wind.str.replace("~"," /")
wind1=wind.str.extract(r"(\S+) ≤3级",expand=False)
wind2=wind.str.extract(r"/(\S+) ≤3级",expand=False)
error_wind1=list(wind1[wind1.isnull()==True].index)
wind1[error_wind1]=weather_data["风力风向"][error_wind1]

error_wind2=list(wind2[wind2.isnull()==True].index)
wind2[error_wind2]=weather_data["风力风向"][error_wind2]
wind1[352]="东南风"
wind2[352]="东北风"

wind1[352]="东风"
wind2[353]="东北风"

wind1[361]="东风"
wind2[361]="东北风"

weather_data["wind1"]=wind1
weather_data["wind2"]=wind2

In [41]:
print(wind1[wind1.str.contains("东风~东南风")==True])
weather_data.loc[353,"wind1"]="东南风"
print(weather_data.loc[353])

353    东风~东南风
Name: 风力风向, dtype: object
accidenttime       2015-12-20
weather                    小雨
temperature           5℃ / 1℃
风力风向                   东风~东南风
temperature_min             1
temperature_max             5
weather1                   小雨
weather2                   小雨
wind1                     东南风
wind2                     东北风
Name: 353, dtype: object


In [43]:
weather_data=weather_data.drop("weather",axis=1)
weather_data=weather_data.drop("风力风向",axis=1)
weather_data=weather_data.drop("temperature",axis=1)
weather_data.temperature_min=weather_data.temperature_min.astype(np.int64)
weather_data.temperature_max=weather_data.temperature_max.astype(np.int64)
diff_temperature=weather_data.temperature_max-weather_data.temperature_min
weather_data["diff_temperature"]=diff_temperature
print(weather_data.head())

  accidenttime  temperature_min  temperature_max weather1 weather2 wind1  \
0     2015-1-1                6                8        阴        阴   东北风   
1     2015-1-2                6               12       多云       多云    南风   
2     2015-1-3                7               13       多云       多云    南风   
3     2015-1-4                7               13       多云        阴   东北风   
4     2015-1-5                3               11       小雨       小雨   东南风   

  wind2  diff_temperature  
0    南风                 2  
1    南风                 6  
2    南风                 6  
3   东南风                 6  
4   东北风                 8  


In [48]:
# print(wind1[wind1.str.contains("南风 ≤3级 /南风 3-4级")==True])
weather_data.loc[177,"wind2"]="南风"
print(weather_data.loc[177])

accidenttime        2015-6-27
temperature_min            23
temperature_max            30
weather1                   多云
weather2                   多云
wind1                      南风
wind2                      南风
diff_temperature            7
Name: 177, dtype: object


In [49]:
weather_data.to_csv("weather_data_processed.csv",encoding="utf-8",index_label="id")